In [1]:
import pandas as pd
import gc

import os
import sys
import urllib.request
import json

In [2]:
df = pd.read_csv("preprocess_data_bank.csv", encoding = 'cp949')

In [3]:
df.head()

,index,은행코드,은행명,점포명,주소_변환,주소,x_axis,y_axis
0,0,30009,기업,본부총괄,서울시 중구 을지로2가 50,서울특별시 중구 을지로2가 50,126.9865697,37.5665076
1,1,30012,기업,영업부,서울특별시 중구 을지로 79,서울특별시 중구 을지로 79 기업은행,126.9865516,37.566501
2,2,30025,기업,동대문（지）,서울특별시 종로구 종로 189 (종로4가),서울특별시 종로구 종로 189 IBK기업은행,126.9987894,37.5710883
3,3,30038,기업,상계역（지）,서울특별시 노원구 덕릉로 690,서울특별시 노원구 덕릉로 690 기업은행,127.0756768,37.6591873
4,4,30041,기업,수유동（지）,서울특별시 강북구 삼양로 511,서울특별시 강북구 삼양로 511 기업은행,127.0137753,37.648693


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4011 entries, 0 to 4010
Data columns (total 8 columns):
index     4011 non-null int64
은행코드      4011 non-null int64
은행명       4011 non-null object
점포명       4011 non-null object
주소_변환     4011 non-null object
주소        4011 non-null object
x_axis    4011 non-null object
y_axis    4011 non-null object
dtypes: int64(2), object(6)
memory usage: 250.8+ KB


In [5]:
df.drop('index', axis = 1, inplace = True)

In [6]:
df['is_SEOUL'] = df.주소_변환.apply(lambda x : 1 if '서울' in x else 0)

In [7]:
df_seoul = df[df.is_SEOUL == 1]

In [8]:
df_seoul.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1886 entries, 0 to 3998
Data columns (total 8 columns):
은행코드        1886 non-null int64
은행명         1886 non-null object
점포명         1886 non-null object
주소_변환       1886 non-null object
주소          1886 non-null object
x_axis      1886 non-null object
y_axis      1886 non-null object
is_SEOUL    1886 non-null int64
dtypes: int64(2), object(6)
memory usage: 132.6+ KB


In [9]:
df_check_each = df_seoul[df_seoul.주소 == 'ERROR_CODE'].reset_index()

In [10]:
df_check_each.columns

Index(['index', '은행코드', '은행명', '점포명', '주소_변환', '주소', 'x_axis', 'y_axis',
       'is_SEOUL'],
      dtype='object')

In [11]:
df_check_each.columns = ['index_number',  '은행코드','은행명', '점포명', '주소_변환', '주소', 'x_axis', 'y_axis','is_SEOUL']

In [12]:
df_check_each.head()

,index_number,은행코드,은행명,점포명,주소_변환,주소,x_axis,y_axis,is_SEOUL
0,232,32492,기업,ＣＭＡ영업점,서울 중구 을지로2가 중소기업은행본점 카드사업부,ERROR_CODE,ERROR_CODE,ERROR_CODE,1
1,330,34076,기업,특수영업부（지）,서울 중구 을지2가 50번지 기업은행,ERROR_CODE,ERROR_CODE,ERROR_CODE,1
2,348,34283,기업,제휴영업점,서울 중구 을지로2가 중소기업은행본점,ERROR_CODE,ERROR_CODE,ERROR_CODE,1
3,361,34416,기업,종합상황실２,서울 중구 을지로2가 기업은행 본점,ERROR_CODE,ERROR_CODE,ERROR_CODE,1
4,362,34429,기업,종합상황실３,서울 중구 을지로2가 기업은행 본점,ERROR_CODE,ERROR_CODE,ERROR_CODE,1


In [13]:
use_list = [38,39,40,90,91,101,102,103,116,122,130,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,161,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178]

In [14]:
divider = pd.DataFrame({"index" : use_list, "checker":  True})

In [15]:
df_check_each = pd.merge(df_check_each.reset_index(), divider, on = 'index', how = 'right')

## 코드로 해결이 안되는 180여개의 주소는 직접 분류하고 주소를 입력하였다

In [16]:
df_check_fixed = pd.read_csv("add_fixed.csv", encoding = 'cp949')
df_check_fixed.head()

,주소_변환,주소_변환2
0,서울시 양천구 신정1동 목동신시가지 9단지 관리동내,서울특별시 양천구 목동서로 340 목동신시가지아파트9단지
1,서울 종로구 공평동 한국스탠다드차타드은행 본점 100번지,서울특별시 종로구 종로 47 제일은행본점
2,서울 종로구 공평동 한국스탠다드차타드은행 본점건물,서울특별시 종로구 종로 47 제일은행본점
3,서울 광진구 노유2동 237－10,서울특별시 광진구 뚝섬로 503 국민은행자양동지점
4,서울특별시 성북구 안암동5가고려대이공대 과학도서관지하,서울특별시 성북구 안암로 145 고려대학교 서울캠퍼스


In [17]:
df_check_each.head()

,index,index_number,은행코드,은행명,점포명,주소_변환,주소,x_axis,y_axis,is_SEOUL,checker
0,38,961,234083,ＳＣ제일,목동９단지（지）,서울시 양천구 신정1동 목동신시가지 9단지 관리동내,ERROR_CODE,ERROR_CODE,ERROR_CODE,1,True
1,39,999,235338,ＳＣ제일,ＣＢ 자가점포관리점,서울 종로구 공평동 한국스탠다드차타드은행 본점 100번지,ERROR_CODE,ERROR_CODE,ERROR_CODE,1,True
2,40,1116,238034,ＳＣ제일,글로벌기업금융 ＲＡ,서울 종로구 공평동 한국스탠다드차타드은행 본점건물,ERROR_CODE,ERROR_CODE,ERROR_CODE,1,True
3,90,1581,47762,국민,영동교,서울 광진구 노유2동 237－10,ERROR_CODE,ERROR_CODE,ERROR_CODE,1,True
4,91,1638,48554,국민,고려대학교（출）,서울특별시 성북구 안암동5가고려대이공대 과학도서관지하,ERROR_CODE,ERROR_CODE,ERROR_CODE,1,True


In [18]:
df_check_each.drop('index', axis = 1, inplace = True)
df_check_each = pd.merge(df_check_each.reset_index(), df_check_fixed.reset_index(), on = "index", how = 'left')
df_check_each.drop(['index','checker','주소_변환_x','주소_변환_y'], axis = 1, inplace = True)

In [19]:
df_check_each.columns = ['index_number', '은행코드', '은행명', '점포명', '주소', 'x_axis', 'y_axis', 'is_SEOUL', '주소_변환']

In [20]:
df_check_each.head()

,index_number,은행코드,은행명,점포명,주소,x_axis,y_axis,is_SEOUL,주소_변환
0,961,234083,ＳＣ제일,목동９단지（지）,ERROR_CODE,ERROR_CODE,ERROR_CODE,1,서울특별시 양천구 목동서로 340 목동신시가지아파트9단지
1,999,235338,ＳＣ제일,ＣＢ 자가점포관리점,ERROR_CODE,ERROR_CODE,ERROR_CODE,1,서울특별시 종로구 종로 47 제일은행본점
2,1116,238034,ＳＣ제일,글로벌기업금융 ＲＡ,ERROR_CODE,ERROR_CODE,ERROR_CODE,1,서울특별시 종로구 종로 47 제일은행본점
3,1581,47762,국민,영동교,ERROR_CODE,ERROR_CODE,ERROR_CODE,1,서울특별시 광진구 뚝섬로 503 국민은행자양동지점
4,1638,48554,국민,고려대학교（출）,ERROR_CODE,ERROR_CODE,ERROR_CODE,1,서울특별시 성북구 안암로 145 고려대학교 서울캠퍼스


## 작업한 코드의 좌표 받아오기

In [21]:
client_id = "******"
client_secret = "******"

In [22]:
def address_to_axis(address):
    encText = urllib.parse.quote(address)
    url = "https://openapi.naver.com/v1/map/geocode?query=" + encText # json 결과

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    
    if(rescode==200):
        data_info = json.load(response)
        address_info = data_info['result']['items'][0]['address']
        axis_x = data_info['result']['items'][0]['point']['x']
        axis_y = data_info['result']['items'][0]['point']['y']
        
    else:
        address_info = 'ERROR_CODE'
        axis_x = 'ERROR_CODE'
        axis_y = 'ERROR_CODE'
        
    return (address_info, axis_x, axis_y)

In [23]:
address_list = list()
x_axis_list = list()
y_axis_list = list()

for _ in df_check_each.주소_변환:
    try:
        add_info, x_ax, y_ax = address_to_axis(_)
    except:
        add_info, x_ax, y_ax = ('ERROR_CODE', 'ERROR_CODE', 'ERROR_CODE')
    address_list.append(add_info)
    x_axis_list.append(x_ax)
    y_axis_list.append(y_ax)

In [24]:
df_check_each['주소'] = address_list
df_check_each['x_axis'] = x_axis_list
df_check_each['y_axis'] = y_axis_list

In [25]:
df_check_each.index = df_check_each.index_number.values

In [26]:
df_check_each.drop('index_number', axis = 1, inplace = True)

In [27]:
df_seoul = df_seoul[df_seoul.주소 != "ERROR_CODE"].append(df_check_each)

In [28]:
df_seoul = df_seoul.reset_index()
df_seoul.drop('index', axis = 1, inplace = True)

In [31]:
df_seoul.drop('is_SEOUL', axis = 1, inplace = True)

In [32]:
df_seoul.head()

,x_axis,y_axis,은행명,은행코드,점포명,주소,주소_변환
0,126.9865697,37.5665076,기업,30009,본부총괄,서울특별시 중구 을지로2가 50,서울시 중구 을지로2가 50
1,126.9865516,37.566501,기업,30012,영업부,서울특별시 중구 을지로 79 기업은행,서울특별시 중구 을지로 79
2,126.9987894,37.5710883,기업,30025,동대문（지）,서울특별시 종로구 종로 189 IBK기업은행,서울특별시 종로구 종로 189 (종로4가)
3,127.0756768,37.6591873,기업,30038,상계역（지）,서울특별시 노원구 덕릉로 690 기업은행,서울특별시 노원구 덕릉로 690
4,127.0137753,37.648693,기업,30041,수유동（지）,서울특별시 강북구 삼양로 511 기업은행,서울특별시 강북구 삼양로 511


In [33]:
df_seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1749 entries, 0 to 1748
Data columns (total 7 columns):
x_axis    1749 non-null object
y_axis    1749 non-null object
은행명       1749 non-null object
은행코드      1749 non-null int64
점포명       1749 non-null object
주소        1749 non-null object
주소_변환     1749 non-null object
dtypes: int64(1), object(6)
memory usage: 95.7+ KB


In [38]:
# 서울에 있는 은행지점만 모두 취합 완료하였다.

df_seoul.to_csv("bank_in_SEOUL_with_AXIS.csv", index = False, encoding = 'cp949')

In [39]:
# 잘 저장되었는지 확인
a = pd.read_csv("bank_in_SEOUL_with_AXIS.csv", encoding = 'cp949')
a.head()

,x_axis,y_axis,은행명,은행코드,점포명,주소,주소_변환
0,126.986570,37.566508,기업,30009,본부총괄,서울특별시 중구 을지로2가 50,서울시 중구 을지로2가 50
1,126.986552,37.566501,기업,30012,영업부,서울특별시 중구 을지로 79 기업은행,서울특별시 중구 을지로 79
2,126.998789,37.571088,기업,30025,동대문（지）,서울특별시 종로구 종로 189 IBK기업은행,서울특별시 종로구 종로 189 (종로4가)
3,127.075677,37.659187,기업,30038,상계역（지）,서울특별시 노원구 덕릉로 690 기업은행,서울특별시 노원구 덕릉로 690
4,127.013775,37.648693,기업,30041,수유동（지）,서울특별시 강북구 삼양로 511 기업은행,서울특별시 강북구 삼양로 511


In [40]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1749 entries, 0 to 1748
Data columns (total 7 columns):
x_axis    1749 non-null float64
y_axis    1749 non-null float64
은행명       1749 non-null object
은행코드      1749 non-null int64
점포명       1749 non-null object
주소        1749 non-null object
주소_변환     1749 non-null object
dtypes: float64(2), int64(1), object(4)
memory usage: 95.7+ KB
